This script shows all flights currently on route to Kazan airport

In [1]:
# from my_keys import avaviationstack_key
import requests
import json
import pandas as pd
import folium
#  ключ лежит в папке выше этого скрипта
import sys, os.path
sys.path.append(os.path.abspath('../'))
from my_keys import avaviationstack_key 

In [2]:
def load_page(offset, limit):
    try:
        api_request = requests.get(f"http://api.aviationstack.com/v1/flights?access_key={avaviationstack_key}&arr_icao=UWKD&offset={offset}&limit={limit}")
        reply = json.loads(api_request.content)
    except Exception:
        reply = "Error..."
    #  если мы не выбрали все результаты в последнем запросе, делаем следующий с нужным offset
    if reply['pagination']['offset'] + reply['pagination']['count'] < reply['pagination']['total']:
        next_offset = reply['pagination']['offset'] + reply['pagination']['count']
    else:
        next_offset = None
    return reply['data'], next_offset


offset = 0
result = []

while offset is not None:
    data, offset = load_page(offset=offset, limit=100)
    result.extend(data)

df = pd.DataFrame.from_dict(result)
df['departure_icao'] = [d.get('icao') for d in df.departure]
print(df.head(10))
df.to_excel('UWKD flights.xlsx')

  flight_date flight_status  \
0  2023-02-14     scheduled   
1  2023-02-14     scheduled   
2  2023-02-14     scheduled   
3  2023-02-14     scheduled   
4  2023-02-14     scheduled   
5  2023-02-14     scheduled   
6  2023-02-14     scheduled   
7  2023-02-14     scheduled   
8  2023-02-14     scheduled   
9  2023-02-14     scheduled   

                                           departure  \
0  {'airport': None, 'timezone': None, 'iata': 'L...   
1  {'airport': None, 'timezone': None, 'iata': 'S...   
2  {'airport': None, 'timezone': None, 'iata': 'S...   
3  {'airport': None, 'timezone': None, 'iata': 'S...   
4  {'airport': None, 'timezone': None, 'iata': 'L...   
5  {'airport': None, 'timezone': None, 'iata': 'L...   
6  {'airport': None, 'timezone': None, 'iata': 'M...   
7  {'airport': None, 'timezone': None, 'iata': 'L...   
8  {'airport': None, 'timezone': None, 'iata': 'S...   
9  {'airport': None, 'timezone': None, 'iata': 'E...   

                                         

In [3]:
airports = pd.read_csv("airport_code.csv")
print(airports.head(10))

       id ident           type                                name  \
0    6523   00A       heliport                   Total Rf Heliport   
1  323361  00AA  small_airport                Aero B Ranch Airport   
2    6524  00AK  small_airport                        Lowell Field   
3    6525  00AL  small_airport                        Epps Airpark   
4    6526  00AR         closed  Newport Hospital & Clinic Heliport   
5  322127  00AS  small_airport                      Fulton Airport   
6    6527  00AZ  small_airport                      Cordes Airport   
7    6528  00CA  small_airport             Goldstone /Gts/ Airport   
8  324424  00CL  small_airport                 Williams Ag Airport   
9  322658  00CN       heliport     Kitchen Creek Helibase Heliport   

   latitude_deg  longitude_deg  elevation_ft continent iso_country iso_region  \
0     40.070801     -74.933601          11.0       NaN          US      US-PA   
1     38.704022    -101.473911        3435.0       NaN          US 

In [4]:
df = pd.read_excel('UWKD flights.xlsx', sheet_name='Sheet1')
df = pd.merge(df, airports, how='left', left_on=['departure_icao'], right_on=['ident'])

In [17]:
uwkd_coords = 55.606201171875, 49.278701782227
df_markers = df.loc[df['flight_status'] == 'active', ['municipality', 'latitude_deg', 'longitude_deg', 'arrival']].dropna(axis=0)

In [39]:
map = folium.Map(location=uwkd_coords, zoom_start=4)
folium.Marker(location=uwkd_coords, popup='Kazan').add_to(map)

for index, row in df_markers.iterrows():
    print(row['arrival'].replace("'", '"'))
    print(json.loads(row['arrival'].replace("'", '"')))
    folium.Marker(location=(row['latitude_deg'], row['longitude_deg']), popup=row['municipality']).add_to(map)
    folium.PolyLine([(row['latitude_deg'], row['longitude_deg']), uwkd_coords], weight=2, color="red").add_to(map)

map
# m.save('line_example.html')

{"airport": None, "timezone": None, "iata": "KZN", "icao": "UWKD", "terminal": "1A", "gate": None, "baggage": None, "delay": None, "scheduled": "2023-02-14T20:00:00+00:00", "estimated": "2023-02-14T20:00:00+00:00", "actual": None, "estimated_runway": None, "actual_runway": None}


JSONDecodeError: Expecting value: line 1 column 13 (char 12)